In [ ]:
%matplotlib notebook
import re
import sys
import inspect
from math import sqrt
import numpy as np
from scipy import integrate
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import cnames
from matplotlib import animation
#===================
# handy func
def num(s):
    try:
        return int(s)
    except ValueError:
        return float(s)
#===================
#====
# read the result file
simulation_result_file = open("../../box/dtn_simulation_result/dtnrunninglog.txt", "r")
lines = simulation_result_file.readlines();
simulation_result_file.close()
#====
x_simulation_time = -1
x_nodes = -1
x_schedule_list = [] # element is also a list[source, time, destination]
x_pkt_trace_map = {} # seqno -> [one_hop_snap, one_hop_snap... destination_snap]
# hop_snap = [receive_time, hop_id, dest_id, source_id, seqno, src_generate_time, hop_transmit_time, pkt_type]
# destination_snap = []
#====

#=======
# parse for x_ nodes and x_ simualtion_time
# parse for bundle trace and schedule
for line in lines :
    #[time-0;node-0;line-1553]enter ScheduleTx(), time-+3000000000.0ns,size=345, to node-1
    schedule = re.match(r'node-(\d+\.*\d*);line-(\d+\.*\d*)]enter ScheduleTx(), time-+(\d+\.*\d*)ns,\
                        size=(\d+\.*\d*), to node-(\d+\.*\d*)', line)
    #BundleTrace:node_number_=5;simulation_duration_=802
    simulation_preview = re.match(r'BundleTrace\:node_number_=(\d+\.*\d*);simulation_duration_=(\d+\.*\d*)', line)
    #[time-5.40052;node-1;line-638]NOTE:BundleTrace:good! one bundle recept, it's one hop! bp_header=,destination ip=10.0.0.3,source ip=10.0.0.1,source seqno=139,payload size=345,offset size=345,src time stamp=5,hop time stamp=5,bundle type=BundlePacket
    hop = re.match(r'[time-(\d+\.*\d*);node-(\d+\.*\d*);line-(\d+\.*\d*)]NOTE:BundleTrace:good! one bundle recep\
    t, it\'s one hop! bp_header=,destination ip=10.0.0.(\d+\.*\d*),source ip=10.0.0.(\d+\.*\d*),source seqn\
    o=(\d+\.*\d*),payload size=(\d+\.*\d*),offset size=(\d+\.*\d*),src time st\
    amp=(\d+\.*\d*),hop time stamp=(\d+\.*\d*),bundle type=(.*)', line)
    #[time-3.30744;node-1;line-630]NOTE:BundleTrace:Great! one bundle arrive destination! bp_header=,destination ip=10.0.0.2,source ip=10.0.0.1,source seqno=75,payload size=345,offset size=345,src time stamp=3,hop time stamp=3,bundle type=BundlePacket
    destination = re.match(r'#[time-(\d+\.*\d*);node-(\d+\.*\d*);line-(\d+\.*\d*)]NOTE:BundleTrace:Great! on\
    e bundle arrive destination! bp_header=,destination ip=10.0.0.(\d+\.*\d*),so\
    urce ip=10.0.0.(\d+\.*\d*),source seqno=(\d+\.*\d*),payload size=(\d+\.*\d*),offset size=(\d+\.*\d*),src ti\
    me stamp=(\d+\.*\d*),hop time stamp=(\d+\.*\d*),bundle type=BundlePacket', line)
    if schedule :
        sche_source = nums(schedule.group(1))
        sche_time = nums(schedule.group(3)) / 1000000000.0
        sche_destination = nums(schedule.group(5))
        tmp = [sche_source, sche_time, sche_destination]
        x_schedule_list.append(tmp)
    else if hop :
        hop_receive_time = nums(hop.group(1))
        hop_hop_id = nums(hop.group(2))
        hop_dest_id = nums(hop.group(4)) - int(1)
        hop_source_id = nums(hop.group(5)) - int(1)
        hop_seqno = nums(hop.group(6))
        hop_src_generate_time = nums(hop.group(9))
        hop_hop_transmit_time = nums(hop.group(10))
        if (hop_hop_transmit_time > hop_receive_time) :
            sys.exit()
        hop_pkt_type = hop.group(11)
        tmp = [hop_receive_time, hop_hop_id, hop_dest_id, hop_source_id, hop\
               _seqno, hop_src_generate_time, hop_hop_transmit_time, hop_pkt_type]
        old_list = x_pkt_trace_map[hop_seqno]
        old_list.append(tmp)
        old_list.sort(key=lambda tup: tup[0])
        x_pkt_trace_map[hop_seqno] = old_list
    else if simulation_preview :
        x_nodes = simulation_preview.group(1)
        x_simulation_time = simulation_preview.group(2)
    else if destination :
        dst_receive_time = nums(destination.group(1))
        dst_receive_id = nums(destination.group(2))
        dst_dst_id = nums(destination.group(4)) - int(1)